In [1]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Processed-hh-ru.csv')

In [3]:
education_dict = {
    'secondary': 0,
    'special_secondary': 24,
    'unfinished_higher': 12,
    'bachelor': 48,
    'master': 72,
    'higher': 60,
    'candidate': 108,
    'doctor': 144
}
df['study_months'] = df['education_level'].map(education_dict)

In [4]:
# убираем всех обманщиков с купленными дипломами
df = df[(df.applicant_age * 12 - df.study_months) >= 18 * 12]

In [5]:
df.columns

Index(['topic_id', 'topic_creation_date', 'initial_state', 'final_state',
       'resume_id', 'resume_creation_date', 'profession', 'gender',
       'year_of_birth', 'resume_region', 'education_level', 'expected_salary',
       'relocation_status', 'business_trip_readiness', 'work_schedule',
       'resume_employment_type', 'work_experience_months',
       'resume_skills_list', 'vacancy_id', 'vacancy_creation_date',
       'vacancy_region', 'work_schedule_required', 'vacancy_employment_type',
       'compensation_from', 'vacancy_skills_list', 'applicant_age',
       'successful_topic', 'study_months'],
      dtype='object')

In [6]:
df['matched_employed_type'] = 0
df['matched_schedule'] = 0
df['matched_region'] = 0
for index, row in df.iterrows():
    if row['resume_employment_type'] == row['vacancy_employment_type']:
        df.loc[index, 'macthed_employed_type'] = 1
    if row['work_schedule'] == row['work_schedule_required']:
        df.loc[index, 'macthed_schedule'] = 1
    if row['resume_region'] == row['vacancy_region']:
        df.loc[index, 'matched_region'] = 1
df.business_trip_readiness = df.business_trip_readiness.map({'never': 0, 'sometimes': 1, 'ready': 2})

## Подготовка данных

In [7]:
from sklearn.model_selection import train_test_split
X = np.array(df[[
    'applicant_age',
    'work_experience_months',
    'study_months',
    'gender',
    'matched_employed_type',
    'matched_schedule',
    'matched_region',
    'business_trip_readiness'
]])
y = np.array(df.successful_topic)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
x = []
y1 = []
y2 = []
for k in np.arange(5, 150, 5):
    x.append(k)
    print(f'k = {k}')
    knn = KNeighborsClassifier(k).fit(X_train, y_train)
    y1.append(knn.score(X_train, y_train))
    y2.append(knn.score(X_test, y_test))
plt.title('KNN')
plt.plot(x, y1, label='Train Accuracy')
plt.plot(x, y2, label='Test Accuracy')
plt.grid()
plt.legend()

In [ ]:
from sklearn.linear_model import RidgeClassifier
x = []
y1 = []
y2 = []
for a in np.arange(0, 2, 0.1):
    x.append(a)
    print(a)
    rc = RidgeClassifier(a).fit(X_train, y_train)
    y1.append(rc.score(X_train, y_train))
    y2.append(rc.score(X_test, y_test))
plt.title('RidgeClassifier')
plt.plot(x, y1, label='Train Accuracy')
plt.plot(x, y2, label='Test Accuracy')
plt.grid()
plt.legend()

business_trip_readiness
never        193781
ready        147497
sometimes     72886
Name: count, dtype: int64